In [1]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable


Bad key text.latex.unicode in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://githu

In [3]:
# open metadata with scores and metrics
meta = pd.read_csv('tmp_res/METADATA_CURRENT.txt', sep='\t')

# gencode 25
metadata_pc_g25 = pd.read_csv('tmp_res/metadata_pc_g25.txt', sep='\t')


P_Set = pd.read_csv('tmp_res/SET1.txt', sep='\t')

R_Set = pd.read_csv('tmp_res/SET2.txt', sep='\t')



R_Set['pos_of_codon'] = [int(x.split('; ')[1].split('-')[0])-1 for x in R_Set.Riboseq_Summary.tolist()]

R_Set['codon'] = [x.split('; ')[0] for x in R_Set.Riboseq_Summary.tolist()]

R_Set = R_Set[(~R_Set['tr_id'].str.contains('PAR')) & (R_Set['codon'] != 'ATG')]

R_Set[0:2]

,tr_id,gene,N_term_start1,N_term_end1,len_codons,global_coo_50_and_less,strand,global_coo_primary,PhyloCSF120score,number_of_records,ovlp,Riboseq_Summary,Coverage_value_ext,Proteomics_count_ext,tag,transcript_type,CDS_ratio,pos_of_codon,codon
0,ENST00000379389.4,ISG15,1,151,50.0,chr1:1013424-1013573,+,chr1:1013424-1013573,-2443.9971,111,0;0;0,ATC; 98-152; Rank: 288; cov: 53.85,53.846154,0.0,"basic,appris_principal_1,CCDS",protein_coding,0.327240,97,ATC
1,ENST00000349431.10,UBE2J2,28,220,64.0,chr1:1273666-1273815,-,chr1:1273666-1273857,-1285.8186,78,0;0;0,ACG; 173-221; Rank: 328; cov: 90.91,90.909091,0.0,"basic,appris_principal_1,CCDS",protein_coding,0.762814,172,ACG


In [4]:
R_Set[R_Set['gene'] == 'CTD-2135J3.4']

,tr_id,gene,N_term_start1,N_term_end1,len_codons,global_coo_50_and_less,strand,global_coo_primary,PhyloCSF120score,number_of_records,ovlp,Riboseq_Summary,Coverage_value_ext,Proteomics_count_ext,tag,transcript_type,CDS_ratio,pos_of_codon,codon
163,ENST00000554914.1,CTD-2135J3.4,1,91,30.0,chr8:10839757-10839846,-,chr8:10839757-10839846,-1919.5706,110,0;0;0,ATT; 14-92; Rank: 231; cov: 80.95,80.952381,0.0,"basic,appris_principal_1,readthrough_transcript",protein_coding,1.7002,13,ATT


# GSEA 

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt

In [5]:
print (gp.__version__)

0.10.4


In [6]:
names = gp.get_library_name(database='Human') # default: Human
names[:10]

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021']

In [7]:
len(names)

190

In [8]:
[x if x.find('KEGG') != -1 else None for x in names]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'KEGG_2013',
 'KEGG_2015',
 'KEGG_2016',
 'KEGG_2019_Human',
 'KEGG_2019_Mouse',
 'KEGG_2021_Human',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 N

# EnrichR

### RiboSET 

In [9]:
gene_list_rset = list(R_Set.gene.unique())

In [10]:
enrichR_set = {}

for i, gene_set in enumerate(names):
    
    enr = gp.enrichr(gene_list=gene_list_rset,
                 gene_sets=[gene_set],
                 organism='Human', # don't forget to set organism to the one you desired! e.g. Yeast
                 description='test_name',
                 outdir='data/GeneEnrichment/enrichr_kegg',
                 # no_plot=True,
                 cutoff=0.1 # test dataset, use lower value from range(0,1)
                )
    # select only significant 
    enr_res_sel = enr.results[enr.results['Adjusted P-value'] <= 0.05]
    enrichR_set[gene_set] = enr_res_sel
    
    print (i, gene_set, '# of sets with adj pv < 0.05', enr_res_sel.shape[0])

0 ARCHS4_Cell-lines # of sets with adj pv < 0.05 0
1 ARCHS4_IDG_Coexp # of sets with adj pv < 0.05 21
2 ARCHS4_Kinases_Coexp # of sets with adj pv < 0.05 93
3 ARCHS4_TFs_Coexp # of sets with adj pv < 0.05 263
4 ARCHS4_Tissues # of sets with adj pv < 0.05 2
5 Achilles_fitness_decrease # of sets with adj pv < 0.05 1


2021-11-12 15:47:45,811 Warning: No enrich terms using library Achilles_fitness_increase when cutoff = 0.1


6 Achilles_fitness_increase # of sets with adj pv < 0.05 0
7 Aging_Perturbations_from_GEO_down # of sets with adj pv < 0.05 102
8 Aging_Perturbations_from_GEO_up # of sets with adj pv < 0.05 130
9 Allen_Brain_Atlas_10x_scRNA_2021 # of sets with adj pv < 0.05 50
10 Allen_Brain_Atlas_down # of sets with adj pv < 0.05 0
11 Allen_Brain_Atlas_up # of sets with adj pv < 0.05 0


2021-11-12 15:48:08,951 Warning: No enrich terms using library Azimuth_Cell_Types_2021 when cutoff = 0.1


12 Azimuth_Cell_Types_2021 # of sets with adj pv < 0.05 0
13 BioCarta_2013 # of sets with adj pv < 0.05 3
14 BioCarta_2015 # of sets with adj pv < 0.05 1
15 BioCarta_2016 # of sets with adj pv < 0.05 1
16 BioPlanet_2019 # of sets with adj pv < 0.05 32


...

173 Tissue_Protein_Expression_from_ProteomicsDB # of sets with adj pv < 0.05 0
174 Transcription_Factor_PPIs # of sets with adj pv < 0.05 37


2021-11-12 15:59:11,416 Warning: No enrich terms using library UK_Biobank_GWAS_v1 when cutoff = 0.1


175 UK_Biobank_GWAS_v1 # of sets with adj pv < 0.05 0
176 Virus-Host_PPI_P-HIPSTer_2020 # of sets with adj pv < 0.05 2705
177 VirusMINT # of sets with adj pv < 0.05 3
178 Virus_Perturbations_from_GEO_down # of sets with adj pv < 0.05 47
179 Virus_Perturbations_from_GEO_up # of sets with adj pv < 0.05 21
180 WikiPathway_2021_Human # of sets with adj pv < 0.05 2
181 WikiPathways_2013 # of sets with adj pv < 0.05 8
182 WikiPathways_2015 # of sets with adj pv < 0.05 18
183 WikiPathways_2016 # of sets with adj pv < 0.05 22
184 WikiPathways_2019_Human # of sets with adj pv < 0.05 0
185 WikiPathways_2019_Mouse # of sets with adj pv < 0.05 4


2021-11-12 15:59:53,839 Warning: No enrich terms using library dbGaP when cutoff = 0.1


186 dbGaP # of sets with adj pv < 0.05 0
187 huMAP # of sets with adj pv < 0.05 8
188 lncHUB_lncRNA_Co-Expression # of sets with adj pv < 0.05 3
189 miRTarBase_2017 # of sets with adj pv < 0.05 134


In [11]:
Aging_Perturbations_from_GEO_down, 
Aging_Perturbations_from_GEO_up
Allen_Brain_Atlas_10x_scRNA_2021 
COVID-19_Related_Gene_Sets_2021
DSigDB 
Disease_Perturbations_from_GEO_down 
Disease_Perturbations_from_GEO_up 
Drug_Perturbations_from_GEO_2014 
GeneSigDB

MSigDB_Oncogenic_Signatures 
NCI-60_Cancer_Cell_Lines  
Rare_Diseases_AutoRIF_Gene_Lists 
''''''
Genes_Associated_with_NIH_Grants 
''''''

Human_Gene_Atlas
Jensen_COMPARTMENTS 
InterPro_Domains_2019 
Pfam_Domains_2019

ProteomicsDB_2020 
SILAC_Phosphoproteomics

SysMyo_Muscle_Gene_Sets

TargetScan_microRNA_2017


Transcription_Factor_PPIs

Virus-Host_PPI_P-HIPSTer_2020

lncHUB_lncRNA_Co-Expression

SyntaxError: invalid token (<ipython-input-11-d47c7dfe9eba>, line 4)

In [ ]:
pd.concat([enrichR_set['Aging_Perturbations_from_GEO_up'], enrichR_set['Aging_Perturbations_from_GEO_down']]).to_csv('Supplementary_tables_and_plots/tables/Gene_enrichment_RiboSET_aging.txt', sep='\t', index=False)

In [ ]:
enrichR_set['COVID-19_Related_Gene_Sets_2021'].to_csv('Supplementary_tables_and_plots/tables/Gene_enrichment_RiboSET_covid19.txt', sep='\t', index=False)

In [ ]:
enrichR_set['Rare_Diseases_AutoRIF_Gene_Lists'].to_csv('Supplementary_tables_and_plots/tables/Rare_Diseases_AutoRIF_Gene_Lists_Riboseq.txt', sep='\t', index=False)

In [ ]:
pd.concat([enrichR_set['Disease_Perturbations_from_GEO_down'], enrichR_set['Disease_Perturbations_from_GEO_down']]).to_csv('Supplementary_tables_and_plots/tables/Gene_enrichment_RiboSET_Disease_Perturbations_riboseq.txt', sep='\t', index=False)

there were 2 gene enriment hits with terms in Rare_Diseases_AutoRIF_Gene_Lists: Periventricular leukomalacia and Leukomalacia. 

### PhyloSET

In [ ]:
gene_list_pset = list(P_Set.gene.unique())
len(gene_list_pset)

In [ ]:
enrichR_set_phyloset = {}

for i, gene_set in enumerate(names):
    
    enr = gp.enrichr(gene_list=gene_list_pset,
                 gene_sets=[gene_set],
                 organism='Human', # don't forget to set organism to the one you desired! e.g. Yeast
                 description='test_name',
                 outdir='data/GeneEnrichment/enrichr_kegg',
                 # no_plot=True,
                 cutoff=0.1 # test dataset, use lower value from range(0,1)
                )
    # select only significant 
    enr_res_sel = enr.results[enr.results['Adjusted P-value'] <= 0.05]
    enrichR_set_phyloset[gene_set] = enr_res_sel
    
    print (i, gene_set, '# of sets with adj pv < 0.05', enr_res_sel.shape[0])

In [ ]:
enrichR_set_phyloset['RNA-Seq_Disease_Gene_and_Drug_Signatures_from_GEO'].to_csv('Supplementary_tables_and_plots/tables/Gene_enrichment_PhyloSET_RNA.txt', sep='\t', index=False)

In [ ]:
enrichR_set_phyloset['RNA-Seq_Disease_Gene_and_Drug_Signatures_from_GEO']

In [ ]:
enrichR_set_phyloset['MGI_Mammalian_Phenotype_Level_4_2021']

In [ ]:
enrichR_set_phyloset['Jensen_DISEASES']

In [ ]:
enrichR_set_phyloset['Jensen_TISSUES'].to_csv('Supplementary_tables_and_plots/tables/Gene_enrichment_PhyloSET_Jensen_TISSUES.txt', sep='\t', index=False)

--------

## Plot enrichR 

In [ ]:
sns.scatterplot(data=enr_res_sel, x="Adjusted P-value", y="Combined Score")

In [ ]:
tmp = enrichR_set['COVID-19_Related_Gene_Sets_2021'][['Term', 'Adjusted P-value', 'Combined Score', 'Genes', 'Overlap']]
tmp['hits'] = [len(x.split(';')) for x in tmp['Genes']]
tmp['-log.adj.pv'] = [-1*np.log10(x) for x in tmp['Adjusted P-value'].tolist()]

tmp.head(10) 

In [ ]:
sns.set(font_scale=3)  # crazy big

sns.set_style("white")

fig, ax = plt.subplots(figsize=(10, 5))

ax = sns.scatterplot(x=tmp.head(10)['-log.adj.pv'], 
              y=tmp.head(10).Term, 
             size=tmp.head(10).hits*4,
             hue = tmp.head(10)['Combined Score'])

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#handles, labels = ax.get_legend_handles_labels()

#ax._legend.remove()
#ax.legend(handles, labels, ncol=2, loc='upper center', 
#                bbox_to_anchor=(1.4, 1), frameon=False)

------

In [ ]:
enrichR_set['Jensen_COMPARTMENTS']['Term'].tolist()

In [ ]:
enrichR_set['InterPro_Domains_2019']

Thioredoxin domain (7/40 overlap, adj.p-value = 0.002582), Ubiquitin-conjugating enzyme E2 (6/40 overlap, adj.p-value = 0.000121)

In [ ]:
enrichR_set['Pfam_Domains_2019']

In [ ]:
enrichR_set['DSigDB']

In [ ]:
enrichR_set['Rare_Diseases_AutoRIF_Gene_Lists']

In [ ]:
Aging_Perturbations_from_GEO_down, 
Aging_Perturbations_from_GEO_up
Allen_Brain_Atlas_10x_scRNA_2021 
COVID-19_Related_Gene_Sets_2021
DSigDB 
Disease_Perturbations_from_GEO_down 
Disease_Perturbations_from_GEO_up 
Drug_Perturbations_from_GEO_2014 
GeneSigDB

MSigDB_Oncogenic_Signatures 
NCI-60_Cancer_Cell_Lines  
Rare_Diseases_AutoRIF_Gene_Lists 
''''''
Genes_Associated_with_NIH_Grants 
''''''

Human_Gene_Atlas
Jensen_COMPARTMENTS 
InterPro_Domains_2019 
Pfam_Domains_2019

ProteomicsDB_2020 
SILAC_Phosphoproteomics

SysMyo_Muscle_Gene_Sets

TargetScan_microRNA_2017


Transcription_Factor_PPIs

Virus-Host_PPI_P-HIPSTer_2020

lncHUB_lncRNA_Co-Expression

# GO terms 

In [77]:
# ontology basic file 
# can be retireved by:
# from goatools.base import download_go_basic_obo
# obo_fname = download_go_basic_obo()

# download associations??

# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
#from goatools.base import download_ncbi_associations
#fin_gene2go = download_ncbi_associations()

## Load files 

In [5]:
from goatools.obo_parser import GODag

obodag = GODag("data/GeneEnrichment/go-basic.obo")

data/GeneEnrichment/go-basic.obo: fmt(1.2) rel(2021-07-02) 47,229 GO Terms


In [6]:
from __future__ import print_function
from goatools.base import download_ncbi_associations
from goatools.base import download_go_basic_obo
from goatools.anno.genetogo_reader import Gene2GoReader

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader('data/GeneEnrichment/gene2go', taxids=[9606])

HMS:0:00:05.472549 330,320 annotations, 20,687 genes, 18,684 GOs, 1 taxids READ: data/GeneEnrichment/gene2go 


In [7]:
# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated mouse genes".format(NS=nspc, N=len(id2gos)))

CC 19,424 annotated mouse genes
BP 18,506 annotated mouse genes
MF 18,191 annotated mouse genes


In [8]:
import goatools

print(goatools.__version__)

# pip install goatools



v1.1.6


### Load Background gene set 

#### Open HGNC gene conversion table

In [9]:
ids_df1 = pd.read_csv('data/GeneEnrichment/HGNC_id_to_NCBI_geneID.txt', sep='\t')

In [17]:
# ids_df1 = HGNC gene conversion table
# entrez_gene_ids_df
# ids_df = gene info

#### Open entrez ID metadata table 

In [10]:
entrez_gene_ids_df = pd.read_csv('data/GeneEnrichment/gencode.v38.metadata.EntrezGene', sep='\t', header=None)
entrez_gene_ids_df.columns = ['tr_id', 'entrez_id']

print (entrez_gene_ids_df.merge(metadata_pc_g25[['tr_id', 'gene']], on=['tr_id']).gene.nunique())

15861


#### Open gene_info table from ncbi 

In [11]:
# Such info is stored in https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene_info.gz
# take all human genes 

ids_df = pd.read_csv('data/GeneEnrichment/gene_info', sep='\t', 
                     usecols=['#tax_id', 'GeneID','Symbol'], low_memory=True)

ids_df_sel = ids_df[(ids_df['Symbol'] != 'NEWENTRY') & (ids_df['#tax_id'] == 9606)]

print (ids_df_sel.Symbol.nunique(), ids_df_sel.GeneID.nunique())

63404 63413


------

#### Transform gene symbols from gencode25 to entrez gene id (with some missing genes)

In [12]:
ids_df_sel_g25 = ids_df_sel[ids_df_sel['Symbol'].isin(metadata_pc_g25.gene.tolist())].drop_duplicates()

ids_df_sel_g25_LI = ids_df_sel_g25.GeneID.tolist()

remaining = list(set(metadata_pc_g25.gene.tolist()) - set(ids_df_sel_g25.Symbol.tolist()))
print (len(remaining))

1503


In [13]:
ids_df3 = ids_df1[ids_df1['Previous symbols'].notna()]

prev_ids = pd.concat([pd.Series(row['NCBI Gene ID(supplied by NCBI)'], row['Previous symbols'].split(','))              
                    for _, row in ids_df3.iterrows()]).reset_index()

add_548_missing_df = prev_ids[prev_ids['index'].isin(remaining)]
add_548_missing_LI = prev_ids[prev_ids['index'].isin(remaining)][0].tolist()

In [14]:
remaining2 = set(remaining) -  set(prev_ids[prev_ids['index'].isin(remaining)]['index'].tolist())
print (len(remaining2))

957


In [15]:
ids_df2 = ids_df1[ids_df1['Alias symbols'].notna()]

alias_ids = pd.concat([pd.Series(row['NCBI Gene ID(supplied by NCBI)'], row['Alias symbols'].split(','))              
                    for _, row in ids_df2.iterrows()]).reset_index()

add_10_missing2_LI = alias_ids[alias_ids['index'].isin(remaining2)][0].tolist()

In [16]:
remaining3 = set(remaining2) - set(alias_ids[alias_ids['index'].isin(remaining2)]['index'].tolist())

In [17]:
save_remaining3_df = metadata_pc_g25[metadata_pc_g25['gene'].isin(remaining3)].merge(entrez_gene_ids_df, on='tr_id')[['entrez_id', 'gene', 'tr_id']]

save_remaining3_LI = save_remaining3_df.entrez_id.tolist()

print (len(save_remaining3_LI))

remaining4 =  set(remaining3) - set(save_remaining3_df.gene.tolist())

print (len(remaining4))

863
684


---------

In [18]:
total_found = list(set(ids_df_sel_g25_LI + add_548_missing_LI + add_10_missing2_LI + save_remaining3_LI))
print (len(total_found))
print (metadata_pc_g25.gene.nunique())

19621
20330


In [27]:
20330-19621

709

### transform gene symbols intp entrez gene ids

In [19]:
ids_df_sel[0:2]

,#tax_id,GeneID,Symbol
8500240,9606,1,A1BG
8500241,9606,2,A2M


In [20]:
Rset_entrez_ids = ids_df_sel[ids_df_sel['Symbol'].isin(R_Set['gene'].tolist())].GeneID.tolist()

list(set(R_Set.gene.tolist()) -  set(ids_df_sel[ids_df_sel['Symbol'].isin(R_Set['gene'].tolist())].Symbol.tolist()))

['FAM64A',
 'HN1',
 'FAM160B1',
 'AC013461.1',
 'CTD-2135J3.4',
 'H1FX',
 'EPRS',
 'CTGF',
 'AARS',
 'UFD1L',
 'C9orf16',
 'FAM92A1',
 'KIF1BP',
 'YARS',
 'MGEA5',
 'C18orf8',
 'RRNAD1',
 'LARS',
 'TMEM189',
 'NARS']

In [21]:
# ids_df1[ids_df1['Previous symbols'].isin(list(set(R_Set.gene.tolist()) -  set(Rset_entrez_ids.Symbol.tolist())))]

d = {'NARS':4677,
 'CTD-2135J3.4':7013, #PIN2 TERF
 'CTGF':1490,
 'C18orf8':29919, 
 'HN1':51155,
 'LARS':51520,
 'MGEA5':10724,
 'KIF1BP':26128, 
 'C9orf16': 79095,
 'FAM64A':54478,
 'RRNAD1': 51093,
 'YARS':8565,
 'AC013461.1':51776, #ZAK
 'H1FX':8971,
 'AARS':16,
 'FAM92A1':137392, 
 'FAM160B1':57700,  
 'EPRS':2058,   
 'TMEM189':387521,
 'UFD1L':7353}

In [22]:
ids_df2[ids_df2['Alias symbols'].str.contains('PIN2')]

,HGNC ID,Approved symbol,Approved name,Status,Previous symbols,Alias symbols,Chromosome,Accession numbers,RefSeq IDs,NCBI Gene ID(supplied by NCBI)
9304,HGNC:15932,EPPIN,epididymal peptidase inhibitor,Approved,SPINLW1,"EPPIN1, EPPIN2, EPPIN3, dJ461P17.2, WAP7, WFDC...",20q13.12,AF286370,NaN,57119.0
42299,HGNC:11728,TERF1,telomeric repeat binding factor 1,Approved,TRBF1,"PIN2, TRF1, TRF",8q21.11,U74382,NM_017489,7013.0


In [23]:
ids_df3 = ids_df1[ids_df1['Previous symbols'].notna()]

ids_df3[ids_df3['Previous symbols'].str.contains('AC013461.1')]

,HGNC ID,Approved symbol,Approved name,Status,Previous symbols,Alias symbols,Chromosome,Accession numbers,RefSeq IDs,NCBI Gene ID(supplied by NCBI)


In [24]:
Rset_entrez_ids_full = Rset_entrez_ids + list(d.values())

In [25]:
len(Rset_entrez_ids), len(Rset_entrez_ids_full)

(370, 390)

In [26]:
rset_convert_dict = dict(zip(ids_df_sel[ids_df_sel['Symbol'].isin(R_Set['gene'].tolist())].GeneID.tolist(), ids_df_sel[ids_df_sel['Symbol'].isin(R_Set['gene'].tolist())].Symbol.tolist()))


d1 = {v: k for k, v in d.items()}

rset_convert_dict.update(d1)

print (len(list(rset_convert_dict.keys())))

390


### Initialize a GOEA object


In [27]:
GeneID_human = total_found # our gencode25 gene database of converted entrez GeneID
print (len(GeneID_human))

19621


In [28]:
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
import fisher

goeaobj = GOEnrichmentStudyNS(
        GeneID_human, # List of human protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.1, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


Load BP Gene Ontology Analysis ...
 86% 16,954 of 19,621 population items found in association

Load CC Gene Ontology Analysis ...
 92% 18,070 of 19,621 population items found in association

Load MF Gene Ontology Analysis ...
 89% 17,433 of 19,621 population items found in association


### Run Gene Ontology Enrichment Analysis 

In [29]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
geneids_study = Rset_entrez_ids_full

goea_results_all = goeaobj.run_study(geneids_study)

goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run BP Gene Ontology Analysis: current study set of 390 IDs ... 94%    364 of    389 study items found in association
100%    389 of    390 study items found in population(19621)
Calculating 12,337 uncorrected p-values using fisher
  12,337 GO terms are associated with 16,954 of 19,621 population items
   2,029 GO terms are associated with    364 of    390 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.1=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 390 IDs ... 97%    376 of    389 study items found in association
100%    389 of    390 study items found in population(19621)
Calculating 1,753 uncorrected p-values using fisher
   1,753 GO terms are associated with 18,070 of 19,621 population items
     456 GO terms are associated with    376 of    390 study items
  METH

/home/alla/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### significant results 

In [39]:
print('{N} of {M:,} results were significant'.format(
    N=len(goea_results_sig),
    M=len(goea_results_all)))

13 of 18,519 results were significant


### enriched vs purified results 

In [40]:
print('Significant results: {E} enriched, {P} purified'.format(
    E=sum(1 for r in goea_results_sig if r.enrichment=='e'),
    P=sum(1 for r in goea_results_sig if r.enrichment=='p')))

Significant results: 13 enriched, 0 purified


### Significant GOEA results by namespace

In [41]:
import collections as cx
ctr = cx.Counter([r.NS for r in goea_results_sig])
print('Significant results[{TOTAL}] = {BP} BP + {MF} MF + {CC} CC'.format(
    TOTAL=len(goea_results_sig),
    BP=ctr['BP'],  # biological_process
    MF=ctr['MF'],  # molecular_function
    CC=ctr['CC'])) # cellular_component

Significant results[13] = 0 BP + 4 MF + 9 CC


### Plot significant GO terms (likely to be messy)

In [42]:
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

plot_results("nbt3102_{NS}.png", goea_results_sig)

    9 usr  25 GOs  WROTE: nbt3102_CC.png
    4 usr  12 GOs  WROTE: nbt3102_MF.png


# Transform significant results into table

In [43]:
li = []

for el in goea_results_sig:
    go_term  = el.GO
    meaning = el.name
    hits_sample = '/'.join([str(x) for x in list(el.ratio_in_pop)])
    hits_bg = '/'.join([str(x) for x in list(el.ratio_in_study)])
    pv = el.p_fdr_bh
    genes_ids = list(el.study_items)  
    gene_symbols = list(map(rset_convert_dict.get, genes_ids))
    
    if el.NS == 'BP':
        ns = 'biological_process'
    elif el.NS == 'MF':
        ns = 'molecular_function'
    elif el.NS == 'CC':
        ns = 'cellular_component'
        
    level = el.goterm.level
    
    
    li.append([go_term, ns, meaning, level, hits_sample, hits_bg, pv, ', '.join([str(x) for x in genes_ids]), 
               ', '.join([str(x) for x in gene_symbols])])

In [44]:
rset_go_significant = pd.DataFrame(li, columns = ['GO_term', 'category', 'name', 'level',  'hits_sample', 'hits_background', 'pv_adj_fdr', 'NCBIgeneIDs', 'gene_symbols'])

rset_go_significant.to_csv('Supplementary_tables_and_plots/tables/rset_GO_significant.txt', sep='\t', index=False)

In [45]:
nucleoplasm = rset_go_significant[rset_go_significant['name'] == 'nucleoplasm'].iloc[0].gene_symbols.split(', ')
cytoplasm = rset_go_significant[rset_go_significant['name'] == 'cytoplasm'].iloc[0].gene_symbols.split(', ')
membrane = rset_go_significant[rset_go_significant['name'] == 'membrane'].iloc[0].gene_symbols.split(', ')

print (len(nucleoplasm), len(cytoplasm), len(membrane))
print (len(set(cytoplasm).intersection(nucleoplasm)))
print (len(set(cytoplasm).intersection(membrane)))
print (len(set(nucleoplasm).intersection(membrane)))

print (len(set(nucleoplasm).intersection(membrane).intersection(cytoplasm)))

132 139 69
58
29
16
9


In [49]:
132-7-9-49, 69-7-20-9, 139-9-49-20

(67, 33, 61)

In [46]:
rna_binding = rset_go_significant[rset_go_significant['name'] == 'RNA binding'].iloc[0].gene_symbols.split(', ')
protein_binding = rset_go_significant[rset_go_significant['name'] == 'protein binding'].iloc[0].gene_symbols.split(', ')

print (len(rna_binding), len(protein_binding))
print (len(set(protein_binding).intersection(rna_binding)))

59 323
51


Interestingly, 58 commons genes were found in enriched gene sets associated with term 'nucleoplasm' (132 genes) and 'cytoplasm' (139 genes). 

In [ ]:
rset_go_significant

# PhyloSET 

### transform gene symbols into gene IDs 

In [30]:
# ids_df1 = HGNC gene conversion table
# entrez_gene_ids_df
# ids_df = gene info

ids_df2 = ids_df1[ids_df1['Previous symbols'].notna()]
ids_df3 = ids_df1[ids_df1['Alias symbols'].notna()]

In [31]:
pset_gene_conversion = ids_df[(ids_df['Symbol'].isin(P_Set.gene.tolist())) & (ids_df['#tax_id'] == 9606)][['GeneID', 'Symbol']].drop_duplicates()

In [32]:
remaining  = list(set(P_Set.gene.tolist()) - set(pset_gene_conversion.Symbol.tolist()))
remaining

['GDF5OS', 'H1FX', 'TBC1D29', 'AC007906.1']

In [ ]:
d = {8971: 'H1FX', 
    # '': 'AC007906.1',  # RP11-295M3 # ENST00000624244.1 (AC007906.1)
    554250: 'GDF5OS', 
    26083: 'TBC1D29'}
d

In [ ]:
pset_gene_conversion_dict = dict(zip(pset_gene_conversion.GeneID.tolist(), pset_gene_conversion.Symbol.tolist()))

pset_gene_conversion_dict.update(d)

pset_gene_conversion_li = list(pset_gene_conversion_dict.keys())

print (len(list(pset_gene_conversion_dict.keys())))

### GO enrichment for PhyloSET

In [ ]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
geneids_study = pset_gene_conversion_li

goea_results_all = goeaobj.run_study(geneids_study)

goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]

In [ ]:
print('{N} of {M:,} results were significant'.format(
    N=len(goea_results_sig),
    M=len(goea_results_all)))

print('Significant results: {E} enriched, {P} purified'.format(
    E=sum(1 for r in goea_results_sig if r.enrichment=='e'),
    P=sum(1 for r in goea_results_sig if r.enrichment=='p')))

import collections as cx
ctr = cx.Counter([r.NS for r in goea_results_sig])
print('Significant results[{TOTAL}] = {BP} BP + {MF} MF + {CC} CC'.format(
    TOTAL=len(goea_results_sig),
    BP=ctr['BP'],  # biological_process
    MF=ctr['MF'],  # molecular_function
    CC=ctr['CC'])) # cellular_component

from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

plot_results("Pset_nbt3102_{NS}.png", goea_results_sig)

In [ ]:
goea_results_sig

# GO enrichment for RiboSET_ext

In [34]:
R_Set_ext = pd.read_csv('revision/Supplementary_tables/Supplementary_table_2_RiboSET_ext.txt', sep='\t')

R_Set_ext[0:4]

,tr_id1,Z_score,CDS_coverage,NTE_coverage,CDS_nonzero_fraction,NTE_nonzero_fraction,gene_ext,Start_ext,Stop_ext,global_rank_ext,...,max_sum_overlap_g38_all_exons_strandless,max_sum_overlap_refseq_all_exons,max_sum_overlap_refseq_all_exons_strandless,ovlp,ovlp_strandless,ovlp2,ovlp2_strandless,PhyloCSF120score,number_of_records,records_names
0,ENST00000379454,0.032101,298.318841,317.934783,0.332894,0.260870,ASPH,144,189,1,...,0,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-1473.5792,93,Human; Chimp; Bonobo; Gorilla; Orangutan; Gibb...
1,ENST00000216281,0.139750,195.778990,441.650000,0.158709,0.194444,HSP90AA1,168,207,2,...,0,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-1331.1851,63,Human; Chimp; Gorilla; Orangutan; Gibbon; Crab...
2,ENST00000409753,-0.352664,750.896774,295.191489,0.331183,0.262411,HN1,242,287,4,...,0,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-2973.9610,114,Human; Chimp; Bonobo; Gorilla; Gibbon; Rhesus;...
3,ENST00000321358,0.295986,156.987692,769.086957,0.062564,0.309353,YBX1,44,140,5,...,0,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-204.2880,107,Human; Chimp; Bonobo; Gorilla; Orangutan; Gibb...


In [35]:
pset_gene_conversion = ids_df[(ids_df['Symbol'].isin(R_Set_ext.gene.tolist())) & (ids_df['#tax_id'] == 9606)][['GeneID', 'Symbol']].drop_duplicates()

In [36]:
remaining  = list(set(P_Set.gene.tolist()) - set(pset_gene_conversion.Symbol.tolist()))
remaining

['ALDH3B2',
 'KIAA0930',
 'PABPC1L2B',
 'GLTP',
 'SLC35G5',
 'TAF12',
 'SULT1A1',
 'SLC6A1',
 'H1FX',
 'GDF5OS',
 'RANBP1',
 'OAZ3',
 'UBE2A',
 'MAP3K19',
 'GLOD4',
 'PABPC1L2A',
 'POMZP3',
 'ENY2',
 'TBC1D29',
 'AC007906.1',
 'ADCY5',
 'PLEKHG3']

In [37]:
len(pset_gene_conversion)

3304

In [38]:
pset_gene_conversion_dict = dict(zip(pset_gene_conversion.GeneID.tolist(), pset_gene_conversion.Symbol.tolist()))

#pset_gene_conversion_dict.update(d)

pset_gene_conversion_li = list(pset_gene_conversion_dict.keys())

print (len(list(pset_gene_conversion_dict.keys())))

3304


In [39]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
geneids_study = pset_gene_conversion_li

goea_results_all = goeaobj.run_study(geneids_study)

goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run BP Gene Ontology Analysis: current study set of 3304 IDs ... 90%  2,966 of  3,304 study items found in association
100%  3,304 of  3,304 study items found in population(19621)
Calculating 12,337 uncorrected p-values using fisher
  12,337 GO terms are associated with 16,954 of 19,621 population items
   6,804 GO terms are associated with  2,966 of  3,304 study items
  METHOD fdr_bh:
      18 GO terms found significant (< 0.1=alpha) ( 11 enriched +   7 purified): statsmodels fdr_bh
     676 study items associated with significant GO IDs (enriched)
     129 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 3304 IDs ... 95%  3,135 of  3,304 study items found in association
100%  3,304 of  3,304 study items found in population(19621)
Calculating 1,753 uncorrected p-values using fisher
   1,753 GO terms are associated with 18,070 of 19,621 population items
   1,068 GO terms are associated with  3,135 of  3,304 study items
  ME

In [47]:
print('{N} of {M:,} results were significant'.format(
    N=len(goea_results_sig),
    M=len(goea_results_all)))

print('Significant results: {E} enriched, {P} purified'.format(
    E=sum(1 for r in goea_results_sig if r.enrichment=='e'),
    P=sum(1 for r in goea_results_sig if r.enrichment=='p')))

import collections as cx
ctr = cx.Counter([r.NS for r in goea_results_sig])
print('Significant results[{TOTAL}] = {BP} BP + {MF} MF + {CC} CC'.format(
    TOTAL=len(goea_results_sig),
    BP=ctr['BP'],  # biological_process
    MF=ctr['MF'],  # molecular_function
    CC=ctr['CC'])) # cellular_component

from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj

plot_results("Pset_nbt3102_{NS}.png", goea_results_sig)

44 of 18,519 results were significant
Significant results: 25 enriched, 19 purified
Significant results[44] = 13 BP + 16 MF + 15 CC
   13 usr  94 GOs  WROTE: Pset_nbt3102_BP.png
   15 usr  36 GOs  WROTE: Pset_nbt3102_CC.png
   16 usr  40 GOs  WROTE: Pset_nbt3102_MF.png


In [51]:
li = []

for el in goea_results_sig:
    go_term  = el.GO
    meaning = el.name
    hits_sample = '/'.join([str(x) for x in list(el.ratio_in_pop)])
    hits_bg = '/'.join([str(x) for x in list(el.ratio_in_study)])
    pv = el.p_fdr_bh
    genes_ids = list(el.study_items)  

    gene_symbols = list(map(pset_gene_conversion_dict.get, genes_ids))
    
    if el.NS == 'BP':
        ns = 'biological_process'
    elif el.NS == 'MF':
        ns = 'molecular_function'
    elif el.NS == 'CC':
        ns = 'cellular_component'
        
    level = el.goterm.level
    
    
    li.append([go_term, ns, meaning, level, hits_sample, hits_bg, pv, ', '.join([str(x) for x in genes_ids]), 
               ', '.join([str(x) for x in gene_symbols])])

In [52]:
genes_ids

[]

In [55]:
rset_go_significant = pd.DataFrame(li, columns = ['GO_term', 'category', 'name', 'level',  'hits_bg', 
                                                  'hits_sample', 'pv_adj_fdr', 'NCBIgeneIDs', 'gene_symbols'])

rset_go_significant.to_csv('revision/Supplementary_tables/RiboSET_ext_GO.txt', sep='\t', index=False)

In [60]:
rset_go_significant[rset_go_significant['hits_sample'] != '0/3304'].groupby(by='category').count()

,GO_term,name,level,hits_bg,hits_sample,pv_adj_fdr,NCBIgeneIDs,gene_symbols
category,,,,,,,,
biological_process,13,13,13,13,13,13,13,13
cellular_component,13,13,13,13,13,13,13,13
molecular_function,14,14,14,14,14,14,14,14


In [63]:
a = rset_go_significant[rset_go_significant['hits_sample'] != '0/3304']

In [68]:
nucleoplasm = a[a['name'] == 'nucleoplasm'].iloc[0].gene_symbols.split(', ')

In [69]:
cytoplasm = a[a['name'] == 'cytoplasm'].iloc[0].gene_symbols.split(', ')

In [70]:
membrane = a[a['name'] == 'membrane'].iloc[0].gene_symbols.split(', ')

In [72]:
len(set(nucleoplasm).intersection(cytoplasm))

292

In [73]:
len(set(membrane).intersection(cytoplasm))

138

In [74]:
len(set(nucleoplasm).intersection(membrane))

83

In [75]:
len(nucleoplasm), len(membrane), len(cytoplasm)

(771, 426, 904)